In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d3 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)
eval_env = MyEnv({"eval":True})
policy_own_config = (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
                 {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
                  "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,})
policy_enem_config = (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
                 {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
                  "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":False,})
policies_own = {}
policies_enem = {}
for i in range(eval_env.blue_num):
    policies_own["blue_"+str(i)] = policy_own_config
for i in range(eval_env.red_num):
    policies_enem["red_"+str(i)] = policy_enem_config

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"env_config": {"eval":False},
              "num_gpus": 1,"num_workers": 0,"num_gpus_per_worker": 1,"num_envs_per_worker":eval_env.env_max,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
#config_enem = ppo.DEFAULT_CONFIG.copy()
#config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_gpus_per_worker": 1,"num_envs_per_worker":1,
#          "train_batch_size": 600,
#          "batch_mode": "complete_episodes",
#          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
#          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
#          "sgd_minibatch_size": 600, "num_sgd_iter":20,
#          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
#         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

#adversary = ppo.PPOTrainer(config=config_enem,
#                         logger_creator=custom_log_creator(
#                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    #reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    #reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    for i in range(eval_env.blue_num):
        reload_weights(policy_id="blue_"+str(i),trainer=trainer,set_policy_id="blue_"+str(i))

    #save_weights("red_0",trainer)
    #save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)

for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-05-04 13:18:03,268	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-05-04 13:18:03,270	WARNING ppo.py:231 -- `train_batch_size` (30000) cannot be achieved with your other settings

2022-05-04 13:18:20,834	INFO trainable.py:130 -- Trainable.setup took 17.568 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-04 13:18:20,837	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')
rewards = {}
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = False
    eval_env.reset()
    proc_list = [0]*eval_env.env_max
    f = open("proc_list"+".pkl",'wb')
    pickle.dump(proc_list,f)
    f.close()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL,eval_env.blue_num)
    for j in range(eval_env.blue_num):
        save_weights("blue_"+str(j),trainer)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')

        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            #trainer.restore(checkpoint_path=model_path)
            trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))
            for j in range(eval_env.blue_num):
                reload_weights(policy_id="blue_"+str(j),trainer=trainer,set_policy_id="blue_"+str(j))
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            
            state_0=[state_0]*eval_env.blue_num
            action_dict0 = [action_dict0]*eval_env.blue_num
            
            for j in range(eval_env.blue_num):
                rewards["blue_"+str(j)] = 0

            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                for j in range(eval_env.blue_num):
                    action_temp = trainer.compute_single_action(obs["blue_"+str(j)],
                                                                state=state_0[j],prev_action=None,prev_reward=None,
                                                                policy_id="blue_"+str(j),explore=False)
                    state_0[j] = action_temp[1]
                    action_dict["blue_"+str(j)] = action_temp[0]
                obs, rewards, dones, infos = eval_env.step(action_dict)
                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])


                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()

                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    for j in range(eval_env.blue_num):
                        save_weights("blue_"+str(j),trainer)
                    #save_weights("blue_1",trainer)
                    #save_weights("blue_2",trainer)
                    # print(f'all done at {env.steps}')
                    break
                

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
3 126 blue_1 Shoot at red_1 launch distance : 57460.37725250331 True True
3 Same tgt shoot
3 137 blue_1 Shoot at red_0 launch distance : 56785.363959386574 True True
2 145 blue_0 DOWN
1 214 blue_1 Shoot at red_0 launch distance : 58580.012546943006 True True
1 Same tgt shoot
1 226 blue_1 Shoot at red_1 launch distance : 59619.01060903309 True True
3 232 blue_1 Splash :red_1
3 235 blue_1 DOWN
3 240 blue_1 Splash :red_0
3 WIN
3 blue_0 False True 240 15.00152843 14.806095990000001
3 blue_1 False True 240 35.40153449666667 158.77561494954023
--------------

0 1136 blue_1: Destroyed
0 TIME LIMIT LOSE
0 blue_0 False False 1136 -0.10237378000000001 -1.41162178
0 blue_1 False False 1136 -0.13807581333333332 -89.2659698133333
1 54 blue_0 DOWN
-------------------------- Scene: 0 --------------------------
2 199 blue_0 Shoot at red_0 launch distance : 59342.75467991017 True True
3 206 blue_1 Shoot at red_0 launch distance : 59485.24175793522 True True
3 214 blue_0 Shoot at red_0 launch distance : 58831.39149297762 True True
2 Same tgt shoot
2 212 blue_0 Shoot at red_1 launch distance : 56493.12115647356 True True
3 222 blue_1 Shoot at red_0 launch distance : 52131.48014395908 True True
3 233 blue_0 Shoot at red_0 launch distance : 51813.740938094794 True True
3 276 red_1 Shoot at blue_1
3 287 red_1 Shoot at blue_1
2 312 blue_0 Splash :red_0
3 316 blue_1 Splash :red_0
2 320 blue_0 Splash :red_1
2 WIN
2 blue_0 False True 320 31.751525773333334 140.08302942230765
2 blue_1 False True 320 13.75152374 12.98458343
-------------------------- Scene: 0 --

2 232 red_1 Shoot at blue_1
0 215 blue_1 Shoot at red_0 launch distance : 58893.16565612686 True True
0 226 blue_1 Shoot at red_0 launch distance : 53196.85156097116 True True
2 300 blue_1 Splash :red_1
2 303 blue_1 Splash :red_0
0 266 red_1 Shoot at blue_1
0 277 red_1 Shoot at blue_1
3 WIN
3 blue_0 False True 1003 11.197937557696909 48.3967018478099
3 blue_1 False True 1003 11.09792905769691 127.45758480567599
-------------------------- Scene: 0 --------------------------
0 328 blue_1 Splash :red_0
0 339 blue_1: Destroyed
2 WIN
2 blue_0 False True 432 12.779335064444446 4.139601984444456
2 blue_1 False True 432 12.779337197777778 146.97244256365306
-------------------------- Scene: 0 --------------------------
3 170 blue_1 Shoot at red_0 launch distance : 59811.83399629207 True True
3 183 blue_1 Shoot at red_0 launch distance : 52647.31488879561 True True
3 213 red_1 Shoot at blue_1
2 119 blue_0 DOWN
3 224 red_1 Shoot at blue_1
2 156 blue_1 Shoot at red_1 launch distance : 59487.03083

2022-05-04 14:07:36,335	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


agent_timesteps_total: 60872
custom_metrics: {}
date: 2022-05-04_14-07-36
done: false
episode_len_mean: 760.9
episode_media: {}
episode_reward_max: 292.7931354766326
episode_reward_mean: 81.42819898084066
episode_reward_min: -91.42598759000008
episodes_this_iter: 40
episodes_total: 40
experiment_id: 47eb63be360f4f448c13169984a3b562
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.727783203125
        entropy_coeff: 0.0
        kl: 0.09895499050617218
        model: {}
        policy_loss: 0.015141483396291733
        total_loss: 327.15875244140625
        vf_explained_var: -0.2145949751138687
        vf_loss: 327.12384033203125
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 5.555573463439941
        entropy_coeff: 0.0
        kl: 0.1718844622373581
        model: {}
    

2022-05-04 14:07:36,558	WARNING ppo.py:231 -- `train_batch_size` (30000) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=4 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 7500.


2022-05-04 14:07:48,858	INFO trainable.py:130 -- Trainable.setup took 12.302 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-04 14:07:48,860	WARNING util.py:55 -- Install gputil for GPU system monitoring.


-------------------------- Scene: 0 --------------------------
4 650 blue_1 Shoot at red_1 launch distance : 59735.446838539676 True True
4 691 red_0 Shoot at blue_0
4 696 blue_0 Shoot at red_1 launch distance : 41433.64414337701 True True
4 702 red_0 Shoot at blue_0
4 707 blue_0 Shoot at red_1 launch distance : 42598.509703979085 True True
4 766 blue_1 Splash :red_1
4 766 blue_0: Destroyed
4 TIME LIMIT LOSE
4 blue_0 False False 1200 0.8997738433333333 66.04085283666669
4 blue_1 False False 1200 0.8937637433333333 64.32319617559598

----------------- Training at steps:1 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
1 135 blue_0 DOWN
2 199 blue_1 Shoot at red_0 launch distance : 59991.99397419626 T

1 317 blue_0 Splash :red_1
1 WIN
1 blue_0 False True 317 31.857996520304937 129.80624467261262
1 blue_1 False True 317 13.788020868990536 5.040065095657198
3 TIME LIMIT LOSE
3 blue_0 False False 1200 0.9938986666666667 -4.623733996666651
3 blue_1 False False 1200 0.8999013333333333 105.82585193454214
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
2 189 blue_1 Shoot at red_0 launch distance : 58988.2324620767 True True
2 200 blue_1 Shoot at red_0 launch distance : 54051.631335233535 True True
0 591 blue_0 Splash :red_0
2 240 red_1 Shoot at blue_1
2 251 red_1 Shoot at blue_1
2 302 blue_1 Splash :red_0
2 309 blue_1: Destroyed
